In [12]:
from openai import OpenAI, AsyncOpenAI
import asyncio
import os
from utils import get_dataset,get_ans_words_chard
from datasets import load_dataset
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
from tqdm import tqdm   
import emoji
import json


In [13]:
def save_results(temp, file_name):

    ## Initialize the file if it does not exist
    if not os.path.exists(file_name):
        with open(file_name,'w') as file:
            json.dump([],file)
    
    file_data = []
    with open(file_name,'r') as file:
        file_data = json.load(file)

    file_data.extend(temp)
    with open(file_name,'w') as file:
        # file.seek(0)
        json.dump(file_data,file)
        

In [14]:
from utils import get_dataset,generate_prompt
import prompts

import numpy as np

errors = 0

model_name = 'gpt-3.5-turbo'
dataset_name = 'boda/guardian_naive_random'
# dataset_name = 'boda/guardian_word_initial_disjoint'

chatgpt_outputs_file = f"results/chatgpt_outputs/{model_name}_{dataset_name.split('/')[-1]}_all_inclusive.json"
base_prompt ='LLAMA3_BASE_PROMPT'
shots = 0
temperature = 0
dataset = get_dataset(dataset_name,
                split='test',
                prompt_key='prompt',
                prompt_head=base_prompt,
                shots=0,
                )

------------------Loading boda/guardian_naive_random/test, and using LLAMA3_BASE_PROMPT, and 0 shot prompt ------------------


Map:   0%|          | 0/28476 [00:00<?, ? examples/s]

In [15]:
dataset

Dataset({
    features: ['input', 'target', 'prompt'],
    num_rows: 28476
})

In [16]:
PROMPT = [{
    'role' : 'system',
    'content': """You are a cryptic crossword expert. The cryptic clue consists of a definition and a wordplay.
The definition is a synonym of the answer and usually comes at the beginning or the end of the clue.
The wordplay gives some instructions on how to get to the answer in another (less literal) way.
The number/s in the parentheses at the end of the clue indicates the number of letters in the answer.
Extract the definiton and the wordplay in the clue, and use them to solve the clue. Finally output the answer on this format:
Answer: <answer>,
""",
},]


# PROMPT = [{
#     'role' : 'system',
#     'content': """You are a cryptic crossword expert. The cryptic clue consists of a definition and a wordplay. The definition is a synonym of the answer and usually comes at the beginning or the end of the clue. The wordplay gives some instructions on how to get to the answer in another (less literal) way. The number/s in the parentheses at the end of the clue indicates the number of letters in the answer.""",
# },
# {
#     'role' : 'user',
#     'content': """### Input 
# agreed toe is deformed somewhat (2,1,6)""",
# },
# {
#     'role' : 'system',
#     'content': """
# ### Respones
# 1. **Recognize the Clue Structure**:
# • Cryptic clues usually consist of a definition and a wordplay. The definition provides a synonym or description of the answer, and the wordplay gives a way to derive it.
# 2. **Identify the Definition**:
# • Look for a part of the clue that could be the definition. Here, “somewhat” is a hint that the answer means “to some extent” or “to a degree”.
# 3. **Analyze the Wordplay**:
# • The phrase “agreed toe is deformed” suggests some manipulation of the letters in “agreed toe”.
# • The word “deformed” is a common indicator of an anagram in cryptic crosswords.
# 4. **Consider the Length Indicator**:
# • The numbers (2,1,6) indicate that the answer is a phrase composed of a 2-letter word, a 1-letter word, and a 6-letter word.
# 5. **Form the Anagram**:
# • Rearrange the letters of “agreed toe” to match the structure (2,1,6).
# • Write out the letters: A, G, R, E, E, D, T, O, E.
# 6. **Try Possible Combinations**:
# • Focus on forming a phrase that fits the pattern (2,1,6) and makes sense in the context of “somewhat”.

# • Break it down:
# • 2-letter word: “to”
# • 1-letter word: “a”
# • 6-letter word: “degree”
# 7. **Verify the Anagram**:

# • Check if “agreed toe” can be rearranged to form “to a degree”.

# • Letters in “agreed toe”: A, G, R, E, E, D, T, O, E

# • Letters in “to a degree”: T, O, A, D, E, G, R, E, E

# 8. **Confirm the Solution**:

# • All letters match, and the phrase “to a degree” means “somewhat”.

# • The phrase “to a degree” fits the length indicator (2,1,6).

# FINAL ANSWER:
# to a degree
# """,
# },
# {
#     'role' : 'user',
#     'content': """### Input 
# Cursed, being literally last in bed (7)""",
# },
# {
#     'role' : 'system',
#     'content': """
# ### Response

# **Step-by-Step Chain of Thoughts:**
# 1. **Recognize the Clue Structure**:

# • The clue typically consists of a definition and wordplay. Here, the definition and wordplay are interwoven.

# 2. **Identify the Definition**:

# • The word “Cursed” can be the definition, meaning the solution should be a synonym for “cursed”.

# 3. **Analyze the Wordplay**:

# • The phrase “being literally last in bed” suggests that there is something related to the position of letters.

# 4. **Break Down the Phrase**:

# • Focus on the word “bed” and consider how “last in bed” could relate to the solution.

# • The word “last” suggests that we are looking at the final letter of “bed”.

# 5. **Final Letter of “bed”**:

# • The final letter of “bed” is “d”.

# 6. **Form the Solution**:

# • The word “blasted” means “cursed” and fits the definition.

# • “Blasted” contains the letter “d” (the last letter of “bed”).

# • The phrase “being literally last in bed” is a hint to include “d” at the end of the word, leading to “blasted”.
# **Conclusion:**

# The solution to the cryptic crossword clue “Cursed, being literally last in bed (7)” is “blasted” because:

# • “Cursed” is the definition of “blasted”.

# • The wordplay “being literally last in bed” suggests taking the final letter “d” from “bed”.

# • The word “blasted” fits the definition and the length indicator (7).

# FINAL ANSWER:
# blasted

# """,
# },
# ]

In [17]:




# num_examples = len(dataset)
num_examples = 100

save_temps = []

offset = 0
## check if there's a file already 
if os.path.exists(chatgpt_outputs_file):
    with open(chatgpt_outputs_file,'r') as file:
        done = json.load(file)
    offset = len(done)  
    print(f"Resuming from {offset} examples")
    if offset == num_examples:
        print("Already done")
        exit(0)
else:
   print(f'Begining a new generation')


# with open(chatgpt_outputs_file, 'a') as f:
for idx ,sample in enumerate(tqdm(dataset.select(range(offset,num_examples)))):
    
    idx = idx + offset
    clue = sample['input']

    target = sample["target"]
    # prompt = sample['prompt']

    prompt = f"""You are a cryptic crossword expert. The cryptic clue consists of a definition and a wordplay.
The definition is a synonym of the answer and usually comes at the beginning or the end of the clue.
The wordplay gives some instructions on how to get to the answer in another (less literal) way.
The number/s in the parentheses at the end of the clue indicates the number of letters in the answer.
Extract the definiton and the wordplay in the clue, and use them to solve the clue. Finally, output the answer on this format:
Answer: <answer>,
Clue:
{clue}
"""
    
    try:
      # correct_answers.append(clue["target"])
      clue_message = {"role": "user", "content": prompt }#clue['prompt']}
      completion = client.chat.completions.create(
        model=model_name,
        messages= [clue_message],
        temperature=temperature
      )

      response = completion.choices[0].message.content.lower()

      # print(response)
      for l in response.split('\n'):
        if 'answer:' in l:
          response = l.split('answer:')[1].strip().replace(',','')
      save_temps.append({'idx': idx, 'clue': clue,'response': response, 'target': target})
    except:
      save_temps.append({'idx': idx})
      errors += 1
      
    if idx % 100 == 0 or idx == num_examples - 1:
      save_results(save_temps,chatgpt_outputs_file)
      save_temps = []


Begining a new generation


100%|██████████| 100/100 [02:00<00:00,  1.21s/it]


In [18]:
import json
with open(chatgpt_outputs_file) as f:
    d = json.load(f)


chatgpt_outputs = []
correct_answers = []
clues = []
errors = 0
for i in d:
    if 'response' in i:
        chatgpt_ans = i['response']#.split('\n')[-1]
        chatgpt_outputs.append(chatgpt_ans)
        correct_answers.append(i['target'])
        clues.append(i['clue'])
    else:
        errors += 1

assert len(chatgpt_outputs) == len(correct_answers)


In [19]:
from calc_scores import calc_and_save_acc
from utils import crop_predictions
import pandas as pd

data_args = pd.DataFrame({
    'dataset': dataset_name,
    'split': 'test',
    'prompt_key': 'prompt',
    'prompt_head': base_prompt,
    'n_shots': shots,
},index=[0])

model_args = pd.DataFrame({
    'model_name_or_path': model_name,
},index=[0])

cleaned_outputs = crop_predictions(clues, chatgpt_outputs)

print(clues,chatgpt_outputs, cleaned_outputs)
calc_and_save_acc(
                chatgpt_outputs, 
                correct_answers, 
                cleaned_predictions= cleaned_outputs, 
                save_file = chatgpt_outputs_file.replace('.json','.txt'), 
                write_outputs = True,
                model_args = model_args,
                data_args= data_args,)

['Achy shaking stopped by iodine, salt and kaolin (5,4)', 'Paid the bail for a sound woman (5)', 'In Germany, thin girls can eat this! (8)', 'Old official partial to free verse (5)', "Coarse-sounding son's in the wilderness (7)", 'Turns up in chilly surroundings yet well received (7)', 'Foreign currency racket completely disheartened auditor (5)', 'The fellow apprehending one felon (5)', 'Dames popular with sailors? (6)', "Last year's leaders entertained by regular journalist (6)", 'Butcher breaks bent pan (7)', 'Improved a carving with direction (4,2)', '\'s "a hairy man", speaking in child\'s play (6)', 'Bend down to king in Chesterfield (6)', 'Message left in note, eg "Castle in the morning" (8)', "A weightwatchers' congress? (4)", 'Dim clue reset by compiler finally for "cimbalom" (8)', 'In the chair, restraining Democrat under influence of drugs (7)', 'Horny beast imagined finding garment ripped, models having fled (7)', 'Sword giving enemy cut to one leg (4)', 'Very cutting, but 